In [1]:
import os
os.chdir("../")

import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
from api.api_connect import fetch_api_data
import psycopg2 
from helpers import load_config, generate_insert_statements, write_insert_statements_to_file, run_script_file, load_data_file

config = load_config()

In [3]:
events = load_data_file("event_large")

In [4]:
# Initialize an empty list to store venue-state bridge data
venue_state_bridge_list = []

# Loop through each event
for event in events:
    # Extract the venues information from the event
    venues = event["_embedded"]["venues"]
        
    # Loop through each venue for the event
    for venue in venues:
        # Extract the ID of the venue
        venue_id = venue["id"]
        
        # Extract the state code from the nested structure inside the venues
        state_code = venue.get("state", {}).get("stateCode", "")
        
        # Check if the venue-state combination is not already in the list
        if (venue_id, state_code) not in venue_state_bridge_list:
            # Append venue-state bridge information to the list
            venue_state_bridge_list.append({
                "venue_id": venue_id,
                "state_code": state_code
            })

In [5]:
# Create a DataFrame from the venue dictionary
venue_state_code_df = pd.DataFrame(data = venue_state_bridge_list)

In [6]:
venue_state_code_df

,venue_id,state_code
0,KovZpZA7AAEA,NY
1,KovZpZA1J67A,DC
2,KovZpZAJaFkA,CA
3,KovZpZAdEt6A,UT
4,KovZ917ACh0,CA
...,...,...
990,KovZpa3swe,MI
991,ZFr9jZek6F,IL
992,KovZpZAEF76A,CA
993,KovZpa1qje,NH


In [7]:
insert_statements = generate_insert_statements(dataframe=venue_state_code_df, table_name="venue_state_bridge")
write_insert_statements_to_file(insert_statements, script_name="insert_venue_state_bridge")
run_script_file("insert_venue_state_bridge")

Insert statements written to 'sql_scripts/insert_venue_state_bridge.sql' successfully!
SQL script executed successfully!
